In [1]:
import sys
sys.path.append('/home/wangs/rs/lib')
import ff
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
sns.set_palette("pastel")
from multiprocessing import Pool
from tqdm import tqdm
import statsmodels.api as sm
from datetime import datetime
import mplfinance as mpf
from functools import partial

涨幅标准：10cm - 20cm

In [3]:
import psutil
import os
print(u'当前进程的内存使用：%.4f GB' % (psutil.Process(os.getpid()).memory_info().rss / 1024 ** 3) )
info = psutil.virtual_memory()
print(u'电脑总内存：%.4f GB' % (info.total / 1024 ** 3))
print(u'当前使用的总内存占比：', info.percent)
print(u'cpu个数：', psutil.cpu_count())

当前进程的内存使用：3.3614 GB
电脑总内存：1007.4503 GB
当前使用的总内存占比： 68.9
cpu个数： 256


In [3]:
#涨幅标准10cm
data_10 = ff.filter0.loc[:'873833.BJ','20200825':].copy()
filtered_index = data_10.index.str.startswith('30') | data_10.index.str.startswith('68') | data_10.index.str.endswith('BJ')
data_10.loc[filtered_index,:] = np.nan
ff.save('10cm', data_10)

In [4]:
#涨幅标准20cm
data_20 = ff.filter0.loc[:'873833.BJ','20200825':].copy()
filtered_index = data_10.index.str.startswith('30') | data_10.index.str.startswith('68')
data_20.loc[~filtered_index,:] = np.nan
ff.save('20cm', data_20)

In [5]:
#次数标准
#五日首板
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_5 = (up.replace({np.nan:0}).rolling(window = 4,axis = 1,closed = 'left').sum() == 0) * up
up_5 = up_5.replace({0.0:np.nan})
ff.save('5日首板', up_5)

In [6]:
#三十日首板
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_30 = (up.replace({np.nan:0}).rolling(window = 29,axis = 1,closed = 'left').sum() == 0) * up
up_30 = up_30.replace({0.0:np.nan})
ff.save('30日首板', up_30)

In [7]:
#连板 (5连板)
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_5_con = ((up.replace({np.nan:0}).rolling(window = 5,axis = 1).sum() == 5) * (up.replace({np.nan:0}).rolling(window = 6,axis = 1).sum() >= 5))
up_5_con = up_5_con.replace({0.0:np.nan}).replace({False:np.nan,True:1.0})
ff.save('5连板', up_5_con)

In [8]:
#连板 (4连板)
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_4_con = ((up.replace({np.nan:0}).rolling(window = 4,axis = 1).sum() == 4) * (up.replace({np.nan:0}).rolling(window = 5,axis = 1).sum() == 4))
up_4_con = up_4_con.replace({0.0:np.nan}).replace({False:np.nan,True:1.0})
ff.save('4连板', up_4_con)

In [9]:
#连板 (3连板)
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_3_con = ((up.replace({np.nan:0}).rolling(window = 3,axis = 1).sum() == 3) * (up.replace({np.nan:0}).rolling(window = 4,axis = 1).sum() == 3))
up_3_con = up_3_con.replace({0.0:np.nan}).replace({False:np.nan,True:1.0})
ff.save('3连板', up_3_con)

In [10]:
#连板 (2连板)
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_2_con = ((up.replace({np.nan:0}).rolling(window = 2,axis = 1).sum() == 2) * (up.replace({np.nan:0}).rolling(window = 3,axis = 1).sum() == 2))
up_2_con = up_2_con.replace({0.0:np.nan}).replace({False:np.nan,True:1.0})
ff.save('2连板', up_2_con)

In [11]:
up_N_con = (up_2_con.replace({np.nan:0}) * 2 +  up_3_con.replace({np.nan:0}) * 3 + up_4_con.replace({np.nan:0}) * 4 + up_5_con.replace({np.nan:0}) * 5).replace({0:np.nan})
# ff.save('N连板', up_N_con)  这个前5天数据会有问题rolling的

In [12]:
#多日n板（多日五板）
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_5_within = ((up.replace({np.nan:0}).rolling(window = 5,axis = 1).sum() == 5)) * up
up_5_within = up_5_within.replace({0.0:np.nan})
ff.save('多日5板', up_5_within)

In [13]:
#多日n板（多日四板）
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_4_within = ((up.replace({np.nan:0}).rolling(window = 5,axis = 1).sum() == 4)) * up
up_4_within = up_4_within.replace({0.0:np.nan})
ff.save('多日4板', up_4_within)

In [14]:
#多日n板（多日三板）
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_3_within = ((up.replace({np.nan:0}).rolling(window = 5,axis = 1).sum() == 3)) * up
up_3_within = up_3_within.replace({0.0:np.nan})
ff.save('多日3板', up_3_within)

In [15]:
#多日n板（多日二板）
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
up_2_within = ((up.replace({np.nan:0}).rolling(window = 5,axis = 1).sum() == 2)) * up
up_2_within = up_2_within.replace({0.0:np.nan})
ff.save('多日2板', up_2_within)

In [16]:
up_N_within = (up_2_within.replace({np.nan:0}) * 2 +  up_3_within.replace({np.nan:0}) * 3 + up_4_within.replace({np.nan:0}) * 4 + up_5_within.replace({np.nan:0}) * 5).replace({0:np.nan})
ff.save('多日N板', up_N_within)

封板形态

In [4]:
data_10 = ff.filter0.loc[:'873833.BJ','20200825':].copy()
up = ff.read('up').loc[data_10.index,data_10.columns].copy()
open = ff.read('open').loc[data_10.index,data_10.columns].copy()
close = ff.read('close').loc[data_10.index,data_10.columns].copy()
high = ff.read('high').loc[data_10.index,data_10.columns].copy()
low = ff.read('low').loc[data_10.index,data_10.columns].copy()
up_limit = pd.DataFrame(ff.read('up_limit').copy(),index = data_10.index,columns = data_10.columns) * ff.filter0.loc[:'873833.BJ','20200825':'20240305']
down_limit = pd.DataFrame(ff.read('down_limit').copy(),index = data_10.index,columns = data_10.columns)
turnover_rate = ff.read('turnover_rate').loc[data_10.index,data_10.columns].copy()

In [ ]:
#一字板 
up_one = (close == high) * (close == open) * (close == high) * (close == low) * up
up_one = up_one.replace({0.0:np.nan})

In [ ]:
#T字板
up_T = (open == high) * (low < close) * (close == high) * up
up_T = up_T.replace({0.0:np.nan})

In [ ]:
#换手板 up除去一字板和t字板
# up_turnover = (turnover_rate > turnover_rate.shift(1,axis = 1)) * up * up.shift(1,axis = 1)
up_turnover = up_one.replace({np.nan:1,1.0:np.nan}) * up_T.replace({np.nan:1,1.0:np.nan}) * up

In [ ]:
#地天板
up_down = (low == down_limit) * (close == up_limit) * up 

日内特征

In [3]:
stock_index = ff.read('一字板').index
time_index = ff.read('一字板').columns
up = ff.read('up').loc[stock_index,time_index].copy()
open = ff.read('open').loc[stock_index,time_index].copy()
close = ff.read('close').loc[stock_index,time_index].copy()
high = ff.read('high').loc[stock_index,time_index].copy()
low = ff.read('low').loc[stock_index,time_index].copy()
up_limit = pd.DataFrame(ff.read('up_limit').copy(),index = stock_index,columns = time_index) * ff.filter0.loc[stock_index,time_index]

In [ ]:
#是否炸板
broke = (high == up_limit) * (close != up_limit)
broke = broke.replace({True:1.0,False:np.nan})
ff.save('炸板股', broke)

开盘价是否涨停

In [ ]:
up_open = (open == up_limit).replace({True:1.0,False:np.nan})
ff.save('开盘是否涨停', up_open)

炸板时间比例 

In [59]:
up_filter = (high == up_limit)
dict = {}
for i in up_filter.sum(axis = 1)[up_filter.sum(axis = 1)>0].index:
    stock = up_filter.loc[i,:]
    dict[i] = list(stock[stock == True].index)

In [ ]:
up_filter = (close == up_limit)
dict = {}
for i in up_filter.sum(axis = 1)[up_filter.sum(axis = 1)>0].index:
    stock = up_filter.loc[i,:]
    dict[i] = list(stock[stock == True].index)
def stock_broke_ratio(stock):
    ratio = []
    for time in dict[stock]:
        start_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 09:31:00'
        end_time = pd.to_datetime(time).strftime('%Y-%m-%d') + ' 15:00:00'
        data = ff.read_min(stock).loc[start_time:end_time,:]
        daily_up_limit = data.loc[:,'high_limit']
        judge = (data.loc[:,'close'] == daily_up_limit) * (data.loc[:,'high'] == daily_up_limit) * (data.loc[:,'low'] == daily_up_limit)
        broke_judge =  (data.loc[:,'close'] != daily_up_limit) | (data.loc[:,'high'] != daily_up_limit) | (data.loc[:,'low'] != daily_up_limit)
        time_len = (judge.cumsum() > 0)
        broke_time = (broke_judge * time_len).sum()
        # if (broke_time != 0):
        #     ratio.append(broke_time / time_len.sum())
        # else:
        #     ratio.append(1.0)
        ratio.append(broke_time/time_len.sum())
    return ratio
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(stock_broke_ratio,dict.keys()),total = len(dict.keys())))
broke_ratio_factor = pd.DataFrame(np.nan,index = up.index,columns = up.columns)
for i in range(len(res_lst)):
    for j in range(len(res_lst[i])):
        broke_ratio_factor.loc[list(dict.keys())[i],list(dict.values())[i][j]] = res_lst[i][j]

In [ ]:
ff.save('炸板时间比例', broke_ratio_factor)

In [ ]:
broke_ratio_factor.sum().sum()

交叉收益计算

In [ ]:
stock_shape = ['一字板','T字板','换手板','地天板']
stock_feature = ['炸板股','炸板时间比例','第一次封板时刻','最后封板时刻','第一次封板时刻成交量','第一次封板时刻成交量占比','最后封板时刻成交量','最后封板时刻成交量占比','板上成交量','板上成交量占比','开盘是否涨停']

In [ ]:
def daily_K_line(event,n,open,close,high,low):
    # 日均事件K线
    T0open=event*open  # 以T0open作为基准
    pre_close = event * close.shift(axis = 1)
    data=pd.DataFrame(columns=['T'+str(j) for j in range(n)])
    for j in range(n):
        _open=(event*open.shift(-j,axis=1))/pre_close
        _close=(event*close.shift(-j,axis=1))/pre_close
        _low=(event*low.shift(-j,axis=1))/pre_close
        _high=(event*high.shift(-j,axis=1))/pre_close
        lst=[_open.mean().mean(),_close.mean().mean(),_low.mean().mean(),_high.mean().mean()]
        data['T'+str(j)]=lst
    data.index=['open','close','low','high']
    data=data.T
    data.index=pd.to_datetime(['20200101','20200102','20200103','20200104','20200105'])
    # mpf.plot(data,type='candle',style='default')
    return data

In [ ]:
open_ = (ff.read('open') * ff.read('post')).loc[data_10.index,data_10.columns].copy()
close = (ff.read('close')* ff.read('post')).loc[data_10.index,data_10.columns].copy()
high = (ff.read('high')* ff.read('post')).loc[data_10.index,data_10.columns].copy()
low = (ff.read('low')* ff.read('post')).loc[data_10.index,data_10.columns].copy()

In [ ]:
event_dict = {}

In [ ]:
for shape in stock_shape:
    for feature in tqdm(stock_feature):
        shape_data = ff.read(shape).copy()
        feature_data = ff.read(feature).copy()
        low_ = (feature_data < feature_data.quantile(1/3)).replace({False:np.nan,True:1.0})
        event_low =  shape_data * low_
        median_ = ((feature_data >= feature_data.quantile(1/3)) & (feature_data < feature_data.quantile(2/3))).replace({False:np.nan,True:1.0})
        event_median =  shape_data * median_
        high_ = (feature_data >= feature_data.quantile(2/3)).replace({False:np.nan,True:1.0})
        event_high =  shape_data * high_
        datalow = daily_K_line(event_low,5,open_,close,high,low)
        datamedian = daily_K_line(event_median,5,open_,close,high,low)
        datahigh = daily_K_line(event_high,5,open_,close,high,low)
        stat= pd.DataFrame(index = ['low','median','high'],columns = ['mean','median','ratio','开盘收益','日内收益'])
        stat['mean'] = [event_low.sum().mean(),event_median.sum().mean(),event_high.sum().mean()]
        stat['median'] = [event_low.sum().median(),event_median.sum().median(),event_high.sum().median()]
        stat['ratio'] = [(event_low.sum()>0).sum()/event_low.shape[1],(event_median.sum()>0).sum()/event_median.shape[1],(event_high.sum()>0).sum()/event_high.shape[1]]
        stat['开盘收益'] = [(datalow.iloc[1,0] - datalow.iloc[0,1]),(datamedian.iloc[1,0] - datamedian.iloc[0,1]),(datahigh.iloc[1,0] - datahigh.iloc[0,1])]
        stat['日内收益'] = [(datalow.iloc[1,1] - datalow.iloc[1,0]),(datamedian.iloc[1,1] - datamedian.iloc[1,0]),(datahigh.iloc[1,1] - datahigh.iloc[1,0])]
        result_list = [stat]
        key = (shape,feature)
        event_dict[key] = result_list

In [ ]:
import pickle 
with open('/home/wangs/data/feature/event_ret.pk','wb') as f:
    pickle.dump(event_dict,f)

对于最后封板时间进行分组筛选计算次数

In [ ]:
stock_shape = ['一字板','T字板','单次涨停','多次涨停','地天板']
stock_feature = ['炸板股','炸板时间比例','第一次封板时刻','最后封板时刻','第一次封板时刻成交量','第一次封板时刻成交量占比','最后封板时刻成交量','最后封板时刻成交量占比','板上成交量','板上成交量占比','开盘是否涨停']
stage = ['low','median','high']

In [ ]:
a = pd.DataFrame(columns = ['封板形态','日内特征','分组','mean','median','ratio','开盘收益','日内收益'])

In [ ]:
a.loc[len(a.index)] = ['一字板','炸板股','low','1','2','3','4','5']

In [ ]:
def setg(x):
    if x<=1:
        return 0
    elif x<=2:
        return 1
    elif x<=5:
        return 2
    elif x<15:
        return 3
    elif x<30:
        return 4
    elif x<60:
        return 5
    elif x<120:
        return 6
    elif x<180:
        return 7
    elif x<240:
        return 8

In [ ]:
ff.read('最后封板时刻').applymap(setg) == 5

In [ ]:
stat = pd.DataFrame(columns = ['封板形态','日内特征','分组','mean','median','ratio','开盘收益','日内收益'])
for shape in stock_shape:
    for feature in tqdm(stock_feature):
        shape_data = ff.read(shape).copy()
        feature_data = ff.read(feature).copy()
        low_ = (feature_data <= feature_data.quantile(1/3)).replace({False:np.nan,True:1.0})
        event_low =  shape_data * low_
        median_ = ((feature_data > feature_data.quantile(1/3)) & (feature_data < feature_data.quantile(2/3))).replace({False:np.nan,True:1.0})
        event_median =  shape_data * median_
        high_ = (feature_data > feature_data.quantile(2/3)).replace({False:np.nan,True:1.0})
        event_high =  shape_data * high_
        datalow = daily_K_line(event_low,5,open_,close,high,low)
        datamedian = daily_K_line(event_median,5,open_,close,high,low)
        datahigh = daily_K_line(event_high,5,open_,close,high,low)
        stat.loc[len(stat.index)] = [shape,feature,'low',event_low.sum().mean(),event_low.sum().median(),(event_low.sum()>0).sum()/event_low.shape[1],(datalow.iloc[1,0] - datalow.iloc[0,1]),(datalow.iloc[1,1] - datalow.iloc[1,0])]
        stat.loc[len(stat.index)] = [shape,feature,'median',event_median.sum().mean(),event_median.sum().median(),(event_median.sum()>0).sum()/event_median.shape[1],(datamedian.iloc[1,0] - datamedian.iloc[0,1]),(datamedian.iloc[1,1] - datamedian.iloc[1,0])]
        stat.loc[len(stat.index)] = [shape,feature,'high',event_high.sum().mean(),event_high.sum().median(),(event_high.sum()>0).sum()/event_high.shape[1],(datahigh.iloc[1,0] - datahigh.iloc[0,1]),(datahigh.iloc[1,1] - datahigh.iloc[1,0])]

In [ ]:
stat.sort_values('开盘收益',ascending = False)

In [ ]:
stat = pd.DataFrame(columns = ['封板形态','日内特征','分组','mean','median','ratio','开盘收益','日内收益'])
for shape in stock_shape:
    feature = '最后封板时刻'
    shape_data = ff.read(shape).copy()
    feature_data = ff.read(feature).copy().applymap(setg)
    for i in range(0,9):
        num_ = (feature_data == i).replace({False:np.nan,True:1.0})
        event_ =  shape_data * num_
        data = daily_K_line(event_,5,open_,close,high,low)
        stat.loc[len(stat.index)] = [shape,feature,i,event_.sum().mean(),event_.sum().median(),(event_.sum()>0).sum()/event_.shape[1],(data.iloc[1,0] - data.iloc[0,1]),(data.iloc[1,1] - data.iloc[1,0])]

In [ ]:
stat.sort_values('开盘收益',ascending = False)

In [ ]:
factor_names1=['closer', 'openr', 'lowr', 'highr', 'ocret', 'lcret', 'hcret', 'ohret', 'ret5', 'ret10', 'ret20', 'ret30', 'excess5', 'excess10', 'excess20', 'excess30', 'ma5', 'bias5', 'ma10',
               'bias10', 'ma15', 'bias15', 'ma20', 'bias20', 'vwap_ret', 'vwaph_ret', 'll5', 'll15', 'll20', 'll30', 'a0', 'a5', 'a15', 'a30', 'ah5', 'ah10', 'ah15', 'std', 'amt_std', 'vol_std', 
               'amt_mean', 'vol_mean', 'turnover', 'turnover_mean', 'turnover_std', 'turnover_skew', 'turnoverf', 'turnoverf_mean', 'turnoverf_std', 'turnoverf_skew', 'mkv', 'l_mkv', 'lhret', 
               'STR', 'MTR', 'SMTR', 'ILLIQ','close','high','low']
factor_names2=['ARRP', 'ARRP_5d_20mean', 'AmpMod', 'BVol', 'CDPDVP', 'ILLIQ', 'IVoldeCorr', 'IntraDayMom20', 'MTR', 'RCP', 'SMTR', 'SPS', 'STR', 'a30', 
               'afternoon_ratio_1', 'ah15', 'amt', 'amt_ma20', 'amt_mean', 'amt_score0', 'amt_score1', 'amt_score2', 'amt_std', 'amt_std20', 'bias10', 'bias15', 'bias20', 'bias5', 
               'cross_std10', 'cross_std10adj', 'ctrade', 'excess10', 'excess20', 'excess30', 'excess5', 'growth_a', 'growth_b', 'growth_c', 'h_rankamt_corr_10', 'h_rankv_corr_10', 'h_rankv_corr_36', 
               'hcret', 'idiov', 'intro_high80_corr', 'ivr', 'l_mkv', 'mkv', 'mom1', 'mom_spring', 'nl_mkv', 'nl_mom', 'open0931ratio', 'open5ratio_ma10vol', 'rank_2_corr_hv10', 'rank_2_corr_hv20', 'rank_ha_corr_sum', 
               'ret20', 'ret30', 'ret5', 'roc121', 'roc20', 'roc240', 'roc5', 'scr', 'std', 'std20', 'std21', 'std240', 'stddev_cov', 'stddev_cov_right', 'sub', 'sysv', 'tail_ratio_1', 'tailrets0', 'tailrets1', 'tliq', 
               'turn20', 'turnover', 'turnover_mean', 'turnover_scale_z', 'turnover_score_ts_mean', 'turnover_score_ts_std', 'turnover_skew', 'turnover_std', 'turnoverf', 'turnoverf_mean', 'turnoverf_skew', 'turnoverf_std', 
               'up_KURS', 'up_plus_down_KURS', 'vol', 'vol_mean', 'vol_std', 'vol_std20', 'volroc_skew', '滞后日内量价corr']
names= list(np.unique(factor_names1+factor_names2))

In [ ]:
def factor_turnover(factor_name):
    shape = '换手板'
    shape_data = ff.read(shape).copy()
    factor = ff.read(factor_name).loc[:,'20200825':'20240305']
    result = [] 
    feature_1 = ((factor<= factor.quantile(1/5)).replace({True:1,False:np.nan}).shift(1,axis=1)) * shape_data
    feature_2 = (((factor> factor.quantile(1/5))*(factor<factor.quantile(2/5))).replace({True:1,False:np.nan}).shift(1,axis=1)) * shape_data
    feature_3 = (((factor> factor.quantile(2/5))*(factor<factor.quantile(3/5))).replace({True:1,False:np.nan}).shift(1,axis=1)) * shape_data
    feature_4 = (((factor> factor.quantile(3/5))*(factor<factor.quantile(4/5))).replace({True:1,False:np.nan}).shift(1,axis=1)) * shape_data
    feature_5 = ((factor> factor.quantile(4/5)).replace({True:1,False:np.nan}).shift(1,axis=1)) * shape_data
    data1 = daily_K_line(feature_1,5,open_,close,high,low)
    data2 = daily_K_line(feature_2,5,open_,close,high,low)
    data3 = daily_K_line(feature_3,5,open_,close,high,low)
    data4 = daily_K_line(feature_4,5,open_,close,high,low)
    data5 = daily_K_line(feature_5,5,open_,close,high,low)
    result.append([shape,factor_name,1,feature_1.sum().mean(),feature_1.sum().median(),(feature_1.sum()>0).sum()/feature_1.shape[1],(data1.iloc[1,0] - data1.iloc[0,1]),(data1.iloc[1,1] - data1.iloc[0,1])])
    result.append([shape,factor_name,2,feature_2.sum().mean(),feature_2.sum().median(),(feature_2.sum()>0).sum()/feature_2.shape[1],(data2.iloc[1,0] - data2.iloc[0,1]),(data2.iloc[1,1] - data2.iloc[0,1])])
    result.append([shape,factor_name,3,feature_3.sum().mean(),feature_3.sum().median(),(feature_3.sum()>0).sum()/feature_3.shape[1],(data3.iloc[1,0] - data3.iloc[0,1]),(data3.iloc[1,1] - data3.iloc[0,1])])
    result.append([shape,factor_name,4,feature_4.sum().mean(),feature_4.sum().median(),(feature_4.sum()>0).sum()/feature_4.shape[1],(data4.iloc[1,0] - data4.iloc[0,1]),(data4.iloc[1,1] - data4.iloc[0,1])])
    result.append([shape,factor_name,5,feature_5.sum().mean(),feature_5.sum().median(),(feature_5.sum()>0).sum()/feature_5.shape[1],(data5.iloc[1,0] - data5.iloc[0,1]),(data5.iloc[1,1] - data5.iloc[0,1])])
    return result
with Pool(24) as p:
    res_lst = list(tqdm(p.imap(factor_turnover,names),total = len(names)))

In [ ]:
df= pd.DataFrame(np.vstack(res_lst),columns = ['封板形态','因子名','分组','mean','median','ratio','开盘收益','日内收益'] ).sort_values(by = '开盘收益',ascending= False)

In [ ]:
df[df['ratio']!='0.0'].head(20)

In [2]:
ff.read('板上成交量占比')['']

,20200825,20200826,20200827,20200828,20200831,20200901,20200902,20200903,20200904,20200907,...,20240221,20240222,20240223,20240226,20240227,20240228,20240229,20240301,20240304,20240305
000001.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.295367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000002.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000004.SZ,NaN,NaN,NaN,NaN,0.178203,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000006.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873703.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873706.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873726.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873806.BJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
